# DEBUG: NumPy ↔ TensorFlow feature parity

This notebook builds the patch stack **B** and computes features step by step with **NumPy**
and **TensorFlow**, comparing each intermediate using `np.allclose`.


In [ ]:
import numpy as np
import tensorflow as tf
import math, time
from typing import Optional
print('TF:', tf.__version__)
print('GPU:', tf.config.list_physical_devices('GPU'))

from ml_spatialfreq_utils import TrainedModelTF, synth_fringe  # ground truth helpers
print('Imports OK')


In [ ]:
# --- Configure your test ---
NR, NC = 128, 127
g = synth_fringe(NR, NC, w0_x=0.25, w0_y=0.12, modulation=1.0, background=0.0, noise_std=0.01)
N_patch = 10; M_patch = 10; r = N_patch // 2; c = M_patch // 2
feature_name = 'feature_projected_DFT'
M_ROI = np.ones_like(g, dtype=bool)


## Build patch stack `B` using the legacy (NumPy) approach


In [ ]:
def build_B_numpy(g: np.ndarray, M_ROI: np.ndarray, N:int, M:int, r:int, c:int):
    NR, NC = g.shape
    M_borders = np.zeros_like(M_ROI, dtype=bool)
    M_borders[r:NR-(N-r), c:NC-(M-c)] = True
    M_proc = (M_ROI & M_borders)
    ir, jc = np.where(M_proc > 0)
    L = ir.size
    B = np.zeros((N, M, L), dtype=np.float32)
    for k in range(L):
        i, j = ir[k], jc[k]
        top = i - r; left = j - c
        B[:, :, k] = g[top:top+N, left:left+M]
    return B, (M_proc.astype(np.float32), ir, jc)

B_np, (M_proc_np, ir_np, jc_np) = build_B_numpy(g, M_ROI, N_patch, M_patch, r, c)
B_np.shape, M_proc_np.sum()


In [ ]:
def report_allclose(name, A, B, rtol=1e-4, atol=1e-5):
    A_np = A if isinstance(A, np.ndarray) else A.numpy()
    B_np = B if isinstance(B, np.ndarray) else B.numpy()
    ok = np.allclose(A_np, B_np, rtol=rtol, atol=atol, equal_nan=True)
    maxdiff = np.nanmax(np.abs(A_np - B_np)) if A_np.size and B_np.size else np.nan
    print(f"{name:25s} allclose={ok}  max|Δ|={maxdiff:.3e}  shapeA={A_np.shape} shapeB={B_np.shape}")
    return ok


## Step-by-step features (NumPy ground truth)


In [ ]:
def gaussian_window_numpy(N:int, M:int):
    x0 = (M // 2) + 1
    y0 = (N // 2) + 1
    xx = np.arange(1, M+1)
    yy = np.arange(1, N+1)
    XX, YY = np.meshgrid(xx, yy, indexing='xy')
    x = (XX - x0).astype(np.float32); y = (YY - y0).astype(np.float32)
    sigma_x = np.float32(M/3.0); sigma_y = np.float32(N/3.0)
    W2 = np.exp(-0.5*(x**2)/(sigma_x**2) - 0.5*(y**2)/(sigma_y**2)).astype(np.float32)
    return W2, x0, y0

def features_numpy(B: np.ndarray, feature_name:str='feature_projected_DFT'):
    N, M, L = B.shape
    W2, x0, y0 = gaussian_window_numpy(N, M)
    mu_sp = B.mean(axis=(0,1), keepdims=True)
    B0 = B - mu_sp
    Bwin = B0 * W2[..., None]
    if feature_name == 'feature_GV':
        muGV = Bwin.mean(axis=(0,1))
        E2   = (Bwin**2).mean(axis=(0,1))
        sigmaGV = np.sqrt(np.maximum(E2 - muGV**2, 1e-12))
        X = Bwin.transpose(2,0,1).reshape(L, -1)
        X_sel = (X - muGV[:,None]) / sigmaGV[:,None]
        return {'W2':W2,'mu_sp':mu_sp,'B0':B0,'Bwin':Bwin,'X_sel':X_sel, 'x0':x0, 'y0':y0}
    G = np.fft.fft2(Bwin, axes=(0,1))
    G[0,0,:] = 0
    G = np.fft.fftshift(G, axes=(0,1))
    G = G * W2[..., None]
    G_abs = np.abs(G)
    GNorm = G_abs.sum(axis=(0,1))
    start = x0 - 1
    abs_G_sp = G_abs[:, start:, :]
    sumTop    = abs_G_sp[:(y0-1), :, :].sum(axis=(0,1))
    sumBottom = abs_G_sp[y0:, :, :].sum(axis=(0,1))
    isTop = (sumTop > sumBottom)
    keep_top = np.zeros_like(abs_G_sp); keep_top[:(y0-1), :, :] = abs_G_sp[:(y0-1), :, :]
    keep_bot = np.zeros_like(abs_G_sp); keep_bot[y0:, :, :] = abs_G_sp[y0:, :, :]
    abs_sel = np.where(isTop[None,None,:], keep_top, keep_bot)
    if feature_name == 'feature_DFT':
        X = abs_sel.transpose(2,0,1).reshape(L,-1)
        X_sel = X / np.maximum(GNorm, 1e-12)[:,None]
    else:
        G_sp_XP = abs_sel.sum(axis=0)
        G_sp_YP = abs_sel.sum(axis=1)
        X = np.concatenate([G_sp_XP.T, G_sp_YP.T], axis=1)
        X_sel = X / np.maximum(GNorm, 1e-12)[:,None]
    return {'W2':W2,'mu_sp':mu_sp,'B0':B0,'Bwin':Bwin,'G':G,'G_abs':G_abs,'GNorm':GNorm,'abs_G_sp':abs_G_sp,'abs_sel':abs_sel,'X_sel':X_sel,'x0':x0,'y0':y0}

np_res = features_numpy(B_np, feature_name)
print('NumPy features ready')


## Step-by-step features (TensorFlow) — exact mirror


In [ ]:
def gaussian_window_tf(N:int, M:int):
    x0 = (M // 2) + 1
    y0 = (N // 2) + 1
    xx = tf.range(1, M + 1, dtype=tf.int32)
    yy = tf.range(1, N + 1, dtype=tf.int32)
    XX, YY = tf.meshgrid(xx, yy, indexing='xy')
    x = tf.cast(XX - x0, tf.float32)
    y = tf.cast(YY - y0, tf.float32)
    sigma_x = tf.cast(M, tf.float32) / 3.0
    sigma_y = tf.cast(N, tf.float32) / 3.0
    W2 = tf.exp(-0.5*(x**2)/(sigma_x**2) - 0.5*(y**2)/(sigma_y**2))
    return W2, x0, y0

def fftshift2d_tf(G):
    N = tf.shape(G)[0]; M = tf.shape(G)[1]
    return tf.roll(tf.roll(G, shift=N//2, axis=0), shift=M//2, axis=1)

def features_tf(B_np: np.ndarray, feature_name: str='feature_projected_DFT'):
    B = tf.convert_to_tensor(B_np, dtype=tf.float32)
    N = B.shape[0]; M = B.shape[1]; L = B.shape[2]
    W2, x0, y0 = gaussian_window_tf(int(N), int(M))
    mu_sp = tf.reduce_mean(B, axis=(0,1), keepdims=True)
    B0 = B - mu_sp
    Bwin = B0 * W2[..., None]
    if feature_name == 'feature_GV':
        eps_ = tf.constant(1e-12, tf.float32)
        muGV = tf.reduce_mean(Bwin, axis=(0,1))
        E2   = tf.reduce_mean(Bwin*Bwin, axis=(0,1))
        sigmaGV = tf.sqrt(tf.maximum(E2 - muGV*muGV, eps_))
        X = tf.reshape(tf.transpose(Bwin, perm=[2,0,1]), [L, -1])
        X_sel = (X - muGV[:,None]) / sigmaGV[:,None]
        return {'W2':W2, 'mu_sp':mu_sp, 'B0':B0, 'Bwin':Bwin, 'X_sel':X_sel, 'x0':x0, 'y0':y0}
    G = tf.signal.fft2d(tf.cast(Bwin, tf.complex64))
    Lint = tf.shape(G)[2]
    idx = tf.stack([
        tf.zeros([Lint], tf.int32),
        tf.zeros([Lint], tf.int32),
        tf.range(Lint, dtype=tf.int32)
    ], axis=1)
    G = tf.tensor_scatter_nd_update(G, idx, tf.zeros([Lint], G.dtype))
    G = fftshift2d_tf(G)
    G = G * tf.cast(W2[..., None], G.dtype)
    G_abs = tf.abs(G)
    GNorm = tf.reduce_sum(G_abs, axis=(0,1))
    start = x0 - 1
    abs_G_sp = G_abs[:, start:, :]
    sumTop    = tf.reduce_sum(abs_G_sp[0:(y0-1), :, :], axis=(0,1))
    sumBottom = tf.reduce_sum(abs_G_sp[y0:, :, :], axis=(0,1))
    isTop = sumTop > sumBottom
    rows = tf.range(tf.shape(abs_G_sp)[0])
    top_mask = tf.cast(rows[:,None,None] < (y0-1), abs_G_sp.dtype)
    bot_mask = tf.cast(rows[:,None,None] >= y0, abs_G_sp.dtype)
    keep_top = abs_G_sp * top_mask
    keep_bot = abs_G_sp * bot_mask
    abs_sel = tf.where(tf.reshape(isTop, [1,1,-1]), keep_top, keep_bot)
    if feature_name == 'feature_DFT':
        X = tf.reshape(tf.transpose(abs_sel, perm=[2,0,1]), [L, -1])
        X_sel = X / tf.maximum(GNorm, 1e-12)[:,None]
    else:
        G_sp_XP = tf.reduce_sum(abs_sel, axis=0)
        G_sp_YP = tf.reduce_sum(abs_sel, axis=1)
        X = tf.concat([tf.transpose(G_sp_XP), tf.transpose(G_sp_YP)], axis=1)
        X_sel = X / tf.maximum(GNorm, 1e-12)[:,None]
    return {'W2':W2,'mu_sp':mu_sp,'B0':B0,'Bwin':Bwin,'G':G,'G_abs':G_abs,'GNorm':GNorm,'abs_G_sp':abs_G_sp,'abs_sel':abs_sel,'X_sel':X_sel,'x0':x0,'y0':y0}

tf_res = features_tf(B_np, feature_name)
print('TF features ready')


## Compare intermediates with `np.allclose`


In [ ]:
ok = True
ok &= report_allclose('W2',           np_res['W2'], tf_res['W2'])
ok &= report_allclose('mu_sp',        np_res['mu_sp'], tf_res['mu_sp'])
ok &= report_allclose('B0',           np_res['B0'], tf_res['B0'])
ok &= report_allclose('Bwin',         np_res['Bwin'], tf_res['Bwin'])
if feature_name != 'feature_GV':
    ok &= report_allclose('G_abs',    np.abs(np_res['G']), tf_res['G_abs'])
    ok &= report_allclose('GNorm',    np_res['GNorm'], tf_res['GNorm'])
    ok &= report_allclose('abs_G_sp', np_res['abs_G_sp'], tf_res['abs_G_sp'])
    ok &= report_allclose('abs_sel',  np_res['abs_sel'], tf_res['abs_sel'])
ok &= report_allclose('X_sel',        np_res['X_sel'], tf_res['X_sel'])
print('\nALL OK =', bool(ok))


### Notes
- Patch stack `B` is built with the **legacy** NumPy method (loop over valid centers) to match your ground truth exactly.
- Feature steps follow the same order: mean removal → spatial window → FFT → zero DC → fftshift → freq window → magnitude → semiplane & half-plane selection → normalization → feature assembly.
- If any check fails, the cell prints the **max absolute difference** to guide where the mismatch starts.
